In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input 
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import SGD
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model
import random
import cv2
from PIL import Image
import os
from keras.utils import to_categorical
from collections import defaultdict

Using TensorFlow backend.


In [173]:
#parameters
classes = 10
batch_size= 60
image_size = 512
train_total = 2200
validation_total = 550

In [46]:
def CACNN_basic_block(x):
    x = Conv2D(8, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(16, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(32, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = GlobalAveragePooling2D()(x)
    return x

In [47]:
x = Input((image_size,image_size,3))

x1 = Conv2D(4, (3,3), padding='valid')(x)
x1 = BatchNormalization()(x1)
x1 = CACNN_basic_block(x1)

x2 = Conv2D(4, (3,3), padding='valid')(x)
x2 = BatchNormalization()(x2)
x2 = CACNN_basic_block(x2)

x3 = Conv2D(4, (3,3), padding='valid')(x)
x3 = BatchNormalization()(x3)
x3 = CACNN_basic_block(x3)

y = layers.concatenate([x1,x2,x3])
y = Dropout(0.2)(y)
y = Dense(256, activation='relu')(y)
y = Dropout(0.5)(y)
y = BatchNormalization()(y)
y = Dense(128, activation='relu')(y)
y = Dropout(0.5)(y)
y = BatchNormalization()(y)
y = Dense(classes, activation='softmax')(y)

In [48]:
model  = Model(inputs=x,outputs=y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_294 (Conv2D)             (None, 510, 510, 4)  112         input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_300 (Conv2D)             (None, 510, 510, 4)  112         input_8[0][0]                    
__________________________________________________________________________________________________
conv2d_306 (Conv2D)             (None, 510, 510, 4)  112         input_8[0][0]                    
__________________________________________________________________________________________________
batch_norm

In [102]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [196]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 23400 images belonging to 10 classes.
Found 550 images belonging to 10 classes.


In [194]:
def train_gen(train_files, train_classes, batch_size, target_size, imggen):
    min_batch_size = batch_size //10
    train_data = list(zip(train_files, train_classes))
    train_data_dict = defaultdict(list)
    for i in range(len(train_data)):
        train_data_dict[train_data[i][0].split('/')[0]].append(train_data[i])
       
    while(True):
        for i in train_data_dict.keys():
            random.shuffle(train_data_dict[i])
        for start in range(0, len(train_data), min_batch_size):
            image_crop_list = []
            image_classes_list = []
            for j in train_data_dict.keys():
                end = min(start + min_batch_size, len(train_data_dict[j])) 
    
                for i in range(start,end):
                    image_classes_list.append(train_data_dict[j][i][1])
                
                x_batch = [train_data_dict[j][i][0] for i in range(start,end)]
                for i in x_batch:
                    image = mpimg.imread('train/' + i)
                    image_crop_list.append(image)
            
            if(len(image_classes_list)<10):
                break
            image_crop_list = [x/255. for x in image_crop_list]
            image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [193]:
train_crop_generator = train_gen(train_generator.filenames,to_categorical(train_generator.classes),batch_size,image_size,train_datagen) 

ValueError: zero-size array to reduction operation maximum which has no identity

In [192]:
xx = next(train_crop_generator)

FileNotFoundError: [Errno 2] No such file or directory: 'train/Samsung-Galaxy-Note3/(GalaxyN3)76.jpg'

In [168]:
xx[1]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.

In [189]:
for i in range(30):
    
    x = next(train_crop_generator)


0
60
(60, 10)
1
60
(60, 10)
2
60
(60, 10)
3
60
(60, 10)
4
60
(60, 10)
5
60
(60, 10)
6
56
(56, 10)
7
54
(54, 10)
8
48
(48, 10)
9
28
(28, 10)
10
60
(60, 10)
11
60
(60, 10)
12
60
(60, 10)
13
60
(60, 10)
14
60
(60, 10)
15
60
(60, 10)
16
56
(56, 10)
17
54
(54, 10)
18
48
(48, 10)
19
28
(28, 10)
20
60
(60, 10)
21
60
(60, 10)
22
60
(60, 10)
23
60
(60, 10)
24
60
(60, 10)
25
60
(60, 10)
26
56
(56, 10)
27
54
(54, 10)
28
48
(48, 10)
29
28
(28, 10)


In [53]:
model.compile(loss='categorical_crossentropy',
             optimizer=SGD(lr=1e-3),
             metrics=['accuracy'])

In [56]:
callbacks = [ModelCheckpoint(filepath='cacnn_1.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, verbose=1),
#EarlyStopping(monitor='val_loss', patience=20, verbose=1),
CSVLogger('./4-metrics_1.csv'),
TensorBoard(log_dir='logs_cacnn', write_graph=True)]

In [61]:
history = model.fit_generator(
                    train_crop_generator,
                    steps_per_epoch = int(np.ceil(4*train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/100


StopIteration: 

In [12]:
#validation accuracy 86%

In [59]:
model.load_weights('cacnn_1.hdf5')

In [60]:
model.evaluate_generator(validation_generator,int(np.ceil(validation_total/batch_size)))

[1.9245632778514514, 0.33090909204699775]

In [51]:
filenames_list = []
images_list = []
for i in os.listdir('test/temp/'):
    img = Image.open('test/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img))
    

In [52]:
new_array_arr = np.array(images_list)
new_array_arr= new_array_arr/255.

In [55]:
test_preds = model.predict(new_array_arr)

In [56]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(test_preds, axis=1)]

In [63]:
for index,i in enumerate(filenames_list):
    if(i.split('_')[-1]=='manip.tif'):
        predicted_classes[index]='HTC-1-M7'

In [64]:
submission = pd.DataFrame({'fname':filenames_list,'camera':predicted_classes})
submission.to_csv('submission_mobilenet_v2_test.csv', encoding="utf8", index=False)

In [65]:
from IPython.display import FileLink
FileLink('submission_mobilenet_v2_test.csv')

/home/sainath/camera_model/submission_mobilenet_v2_test.csv